In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-zoz3co1i/unsloth_6d2a71745aed468b8be09a1796af48b9
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-zoz3co1i/unsloth_6d2a71745aed468b8be09a1796af48b9
  Resolved https://github.com/unslothai/unsloth.git to commit 600ffe2a175312e7f2fbcdffbe6b99e3df20c417
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 10.6 MB/s eta 0:00:00
   ━━

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length=2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model (
                                              model,
                                              target_modules=["q_proj", "k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj",],
                                              lora_alpha = 16,
                                              lora_dropout = 0,
                                              bias = "none",
                                              use_gradient_checkpointing = "unsloth",
                                              random_state = 3407,
                                              use_rslora = False,
                                              loftq_config = None
                                           )

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt_ip = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appopriately completes the request.

###Instruction:
{}

###Input:
{}

###Response:
{}"""


EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
  instructions = examples["Instruction"]
  inputs = examples["Input"]
  outputs = examples["Response"]

  texts = []
  for instruction, input, output in zip(instructions, inputs, outputs):
    text = alpaca_prompt_ip.format(instruction, input, output) + EOS_TOKEN
    texts.append(text)
  return {"text": texts}




In [ ]:
import os
from pathlib import Path

file_path = Path("data/TrainingData.csv")

if file_path.exists():
  print("Training data exists")
else:
  print("Training data does not exists")

Training data exists


In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="data/TrainingData.csv", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer

from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer (
                      model = model,
                      tokenizer = tokenizer,
                      train_dataset = dataset,
                      dataset_text_field="text",
                      max_seq_length = max_seq_length,
                      dataset_num_proc = 2,
                      packing = False,
                      args = TrainingArguments(
                          per_device_train_batch_size=2,
                          gradient_accumulation_steps=4 ,
                          warmup_steps = 5,
                          max_steps=60,
                          learning_rate = 2e-4,
                          fp16= not is_bfloat16_supported(),
                          bf16=is_bfloat16_supported(),
                          logging_steps=1,
                          optim="adamw_8bit",
                          weight_decay=0.01,
                          lr_scheduler_type="linear",
                          seed=3407,
                          output_dir = "outputs"
                      ),
                      )

Map (num_proc=2):   0%|          | 0/17 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 17 | Num Epochs = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.337500
2,3.322000
3,3.223400
4,2.995800
5,2.556300
6,2.222900
7,1.774400
8,1.346100
9,0.960000
10,0.804200


In [ ]:
FastLanguageModel.for_inference(model)

inputs = tokenizer( [alpaca_prompt_ip.format("Select India from Coutnry Dropdown and check Male for Gender check box","","")],return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
tokenizer.batch_decode(outputs)


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appopriately completes the request.\n\n###Instruction:\nSelect India from Coutnry Dropdown and check Male for Gender check box\n\n###Input:\n\n\n###Response:\n{"object":"dropdown", "action":"select", label:"Country", "data":{"value":"India"} }, {"object":"checkbox", "action":"check", label:"Gender", "data":{"value":"Male"} }<|end_of_text|>']

In [ ]:
model.save_pretrained("spotnuru_model")
tokenizer.save_pretrained("spotnuru_model")

('spotnuru_model/tokenizer_config.json',
 'spotnuru_model/special_tokens_map.json',
 'spotnuru_model/tokenizer.json')

In [ ]:
import shutil
shutil.make_archive("spotnuru_model","zip","spotnuru_model")

'/content/spotnuru_model.zip'

In [ ]:
from google.colab import files

files.download("spotnuru_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>